# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value
print(f'answer_1 = {round(sum, 3)}')

limit = 13
generator = square_root_generator(limit)

for n, sqrt_value in enumerate(generator):
    if n == limit - 1:
        print(f'answer_2 = {round(sqrt_value, 3)}')

answer_1 = 8.382
answer_2 = 3.606


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [10]:
import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

info = pipeline.run(people_1(),
                    table_name="people",
					write_disposition="replace")

print(info)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('\nLoaded tables: ')
display(conn.sql("show tables"))

people = conn.sql("SELECT * FROM people").df()
display(people)

print(f'sum_ages = {people.age.sum()}')

info = pipeline.run(people_2(),
                    table_name="people",
					write_disposition="append")

people = conn.sql("SELECT * FROM people ORDER BY id").df()
display(people)

print(f'sum_ages = {people.age.sum()}')

Pipeline dlt_ipykernel_launcher load step completed in 0.32 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\DiogoCosta\Downloads\workshop-1\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707508670.4707298 is LOADED and contains no failed jobs

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merge        │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707508670.4707298,c3AI7ho1bXv/OA,None
1,2,Person_2,27,City_A,1707508670.4707298,2+pbSZVCoez6+A,None
2,3,Person_3,28,City_A,1707508670.4707298,+eUdYllOwCGmxg,None
3,4,Person_4,29,City_A,1707508670.4707298,2idYsznkVParKg,None
4,5,Person_5,30,City_A,1707508670.4707298,PyNkKVqiX0A2GA,None


sum_ages = 140


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707508670.4707298,c3AI7ho1bXv/OA,None
1,2,Person_2,27,City_A,1707508670.4707298,2+pbSZVCoez6+A,None
2,3,Person_3,28,City_A,1707508670.4707298,+eUdYllOwCGmxg,None
3,3,Person_3,33,City_B,1707508671.4245522,tM7gjO+RVZACOw,Job_3
4,4,Person_4,29,City_A,1707508670.4707298,2idYsznkVParKg,None
5,4,Person_4,34,City_B,1707508671.4245522,S6vhzb5QjM6u4A,Job_4
6,5,Person_5,30,City_A,1707508670.4707298,PyNkKVqiX0A2GA,None
7,5,Person_5,35,City_B,1707508671.4245522,qvAWOPUqpkKF2Q,Job_5
8,6,Person_6,36,City_B,1707508671.4245522,tw0MOsY6j07xUQ,Job_6
9,7,Person_7,37,City_B,1707508671.4245522,rW9mgXaw3SWd1w,Job_7


sum_ages = 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [9]:
info = pipeline.run(people_1(),
                    table_name="people_merge",
					write_disposition="replace")

print(info)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('\nLoaded tables: ')
display(conn.sql("show tables"))

people = conn.sql("SELECT * FROM people_merge").df()
display(people)

info = pipeline.run(people_2(),
                    table_name="people_merge",
					write_disposition="merge",
                    primary_key='id')

people = conn.sql("SELECT * FROM people_merge ORDER BY id").df()
display(people)

print(f'sum_ages = {people.age.sum()}')

Pipeline dlt_ipykernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:///c:\Users\DiogoCosta\Downloads\workshop-1\dlt_ipykernel_launcher.duckdb location to store data
Load package 1707508648.839889 is LOADED and contains no failed jobs

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merge        │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707508648.839889,Y3/6tfz2r1jwtA,None
1,2,Person_2,27,City_A,1707508648.839889,cQHByUxgnPmI0w,None
2,3,Person_3,28,City_A,1707508648.839889,YpONZ/T550FvJg,None
3,4,Person_4,29,City_A,1707508648.839889,P8TTagdnj6I+CQ,None
4,5,Person_5,30,City_A,1707508648.839889,vywqwXkHla7/lw,None


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707508648.839889,Y3/6tfz2r1jwtA,None
1,2,Person_2,27,City_A,1707508648.839889,cQHByUxgnPmI0w,None
2,3,Person_3,33,City_B,1707508649.6736479,qFTiy6NSrNz1FA,Job_3
3,4,Person_4,34,City_B,1707508649.6736479,lYDQH6hQ4XazIg,Job_4
4,5,Person_5,35,City_B,1707508649.6736479,iJ3IOxmo1ga9sw,Job_5
5,6,Person_6,36,City_B,1707508649.6736479,m+y+obTcbzX3tg,Job_6
6,7,Person_7,37,City_B,1707508649.6736479,0fimciz7GGBbOg,Job_7
7,8,Person_8,38,City_B,1707508649.6736479,QneLfjE3s+kInQ,Job_8


sum_ages = 266


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX